In [1]:
#kodk filing that kicked this whole thing off
# https://www.secform4.com/filings/31235/0000891839-20-000226.htm

# Start with Indexing Libraries

In [1]:
#parameter tagged cell- for use with shell script!
year = 2014
quarter = 'QTR1'

In [2]:
import edgar
import pandas as pd 

In [3]:
pd.set_option("display.max_colwidth", 100)

In [4]:

ua = [insert your user agent string]

edgar.download_index('/Users/rileybitterli/Desktop/StockBot_Data/StockBot_Indices', 2005, ua, skip_all_present_except_last=True)



In [16]:
import re

file_path = f'/Users/rileybitterli/Desktop/StockBot_Data/StockBot_Indices/{year}-{quarter}.tsv'


In [17]:
file_path

'/Users/rileybitterli/Desktop/StockBot_Data/StockBot_Indices/2014-QTR1.tsv'

In [11]:
index_df = pd.read_csv(file_path, delimiter= '|', names = ('CIK', 'Name', 'Form_Type', 'Date', 'URL_TXT', 'URL_HTML'))

index_df = index_df[index_df['Form_Type']=='4']

In [12]:
index_df['FileURL'] = 'https://sec.gov/Archives/' + index_df['URL_HTML']
index_df.reset_index(inplace = True)

In [13]:
index_df['Date'] = pd.to_datetime(index_df['Date'])

In [14]:
index_df

,index,CIK,Name,Form_Type,Date,URL_TXT,URL_HTML,FileURL
0,0,1000032,BINCH JAMES G,4,2014-03-03,edgar/data/1000032/0001181431-14-010250.txt,edgar/data/1000032/0001181431-14-010250-index.html,https://sec.gov/Archives/edgar/data/1000032/0001181431-14-010250-index.html
1,17,1000180,SANDISK CORP,4,2014-01-08,edgar/data/1000180/0001242648-14-000001.txt,edgar/data/1000180/0001242648-14-000001-index.html,https://sec.gov/Archives/edgar/data/1000180/0001242648-14-000001-index.html
2,18,1000180,SANDISK CORP,4,2014-01-10,edgar/data/1000180/0001242648-14-000002.txt,edgar/data/1000180/0001242648-14-000002-index.html,https://sec.gov/Archives/edgar/data/1000180/0001242648-14-000002-index.html
3,19,1000180,SANDISK CORP,4,2014-01-29,edgar/data/1000180/0001242648-14-000003.txt,edgar/data/1000180/0001242648-14-000003-index.html,https://sec.gov/Archives/edgar/data/1000180/0001242648-14-000003-index.html
4,20,1000180,SANDISK CORP,4,2014-01-31,edgar/data/1000180/0001242648-14-000004.txt,edgar/data/1000180/0001242648-14-000004-index.html,https://sec.gov/Archives/edgar/data/1000180/0001242648-14-000004-index.html
...,...,...,...,...,...,...,...,...
128962,311687,9984,BARNES GROUP INC,4,2014-02-27,edgar/data/9984/0000009984-14-000033.txt,edgar/data/9984/0000009984-14-000033-index.html,https://sec.gov/Archives/edgar/data/9984/0000009984-14-000033-index.html
128963,311688,9984,BARNES GROUP INC,4,2014-02-28,edgar/data/9984/0000009984-14-000034.txt,edgar/data/9984/0000009984-14-000034-index.html,https://sec.gov/Archives/edgar/data/9984/0000009984-14-000034-index.html
128964,311689,9984,BARNES GROUP INC,4,2014-02-28,edgar/data/9984/0000009984-14-000035.txt,edgar/data/9984/0000009984-14-000035-index.html,https://sec.gov/Archives/edgar/data/9984/0000009984-14-000035-index.html
128965,311690,9984,BARNES GROUP INC,4,2014-03-11,edgar/data/9984/0000009984-14-000036.txt,edgar/data/9984/0000009984-14-000036-index.html,https://sec.gov/Archives/edgar/data/9984/0000009984-14-000036-index.html


In [15]:
first_500 = index_df

# Scraping/Parsing

In [16]:
from selenium import webdriver
import concurrent.futures
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from bs4 import BeautifulSoup 
from random import uniform
from time import sleep
import lxml
from bs4.formatter import HTMLFormatter
import re
import pandas as pd
import pickle
import os
import smtplib
from email.message import EmailMessage
import threading
from multiprocessing import Pool
import more_itertools

In [17]:
def send_email(subject, content):
    msg = EmailMessage()
    msg.set_content(content)
    msg['Subject'] = subject
    msg['From'] = 'halpitsstockbot@gmail.com'
    msg['To'] = 'riley.bitterli@gmail.com'

    # Establish a connection to Gmail
    server = smtplib.SMTP('smtp.gmail.com', 587)
    server.starttls()
    server.login([insert your email address], [inser your password])
    server.send_message(msg)
    server.quit()


In [18]:
#get selenium all defined and set up

In [19]:
chrome_path = ChromeDriverManager().install()

In [ ]:
if "THIRD_PARTY_NOTICES.chromedriver" in chrome_path:
    chrome_path = chrome_path.replace("THIRD_PARTY_NOTICES.chromedriver", "chromedriver")

In [ ]:
s = Service(chrome_path)


In [20]:
chrome_options = Options()
chrome_options.add_argument("--disable-background-timer-throttling")
chrome_options.add_argument("--disable-backgrounding-occluded-windows")
chrome_options.add_argument("--disable-renderer-backgrounding")
chrome_options.add_argument("--incognito")
chrome_options.add_argument("--disable-gpu")  
chrome_options.add_argument("--enable-accelerated-2d-canvas")
chrome_options.add_argument("--ignore-gpu-blacklist")
chrome_options.add_argument("--no-sandbox") 



In [21]:
chrome_options

In [33]:
len(first_500) #not actually first 500- keep around as a variable name out of nostalgia

128967

In [23]:
directory_path = f"/Users/rileybitterli/Desktop/StockBot_Data/Draft_VIII/Pickle_Files/{year}/{quarter}"

In [24]:
#only necessary if you've already run for a bit on a particular quarter
import os
import re

def find_highest_batch_number(directory_path):
    # check if the directory exists
    if not os.path.exists(directory_path):
        print(f"Directory {directory_path} does not exist.")
        return 0

    max_batch_number = -1  # initialize the maximum batch number as -1 to indicate not found

    # loop through each file in the directory
    for filename in os.listdir(directory_path):
        if filename.endswith('.pkl'):
            # extract the batch number from the filename using regex
            match = re.search(r"batch(\d+)", filename)
            if match:
                batch_number = int(match.group(1))
                max_batch_number = max(max_batch_number, batch_number)

    if max_batch_number == -1:
        print("No batch number found in any file.")
        return 0
    else:
        print(f"Highest batch number found: {max_batch_number}")
        return max_batch_number

highest_batch = find_highest_batch_number(directory_path)


Highest batch number found: 32


In [25]:
highest_batch = [highest_batch if highest_batch is not None and highest_batch > 0 else 0]


In [26]:
highest_batch = int(highest_batch[0]) + 1

In [27]:
highest_batch

33

In [28]:
# convert DataFrame columns into a list of tuples
url_date_pairs = list(zip(first_500['FileURL'], first_500['Date']))

# define the chunk size
chunk_size = 1000

# create chunks
chunks = list(enumerate([url_date_pairs[i:i + chunk_size] for i in range(0, len(url_date_pairs), chunk_size)]))

# now, `chunks` is a list of indexed tuples, where each tuple contains an index and a list of up to 100 URL-Date pairings.


In [29]:
import os
import re

def filter_chunks(directory_path, chunks):
    found_batches = []
    
    # check if the directory exists
    if not os.path.exists(directory_path):
        print(f"Directory {directory_path} does not exist.")
        return chunks
    
    # loop through each file in the directory
    for filename in os.listdir(directory_path):
        if filename.endswith('.pkl'):
            # extract the batch number from the filename using regex
            match = re.search(r"batch(\d+)", filename)
            if match:
                # append the found batch number to the list
                batch_number = int(match.group(1))
                found_batches.append(batch_number)
    
    # remove the elements in chunks that correspond to the found batch numbers
    filtered_chunks = [chunk for i, chunk in enumerate(chunks) if i not in found_batches]
    
    return filtered_chunks



result_chunks = filter_chunks(directory_path, chunks)
print(result_chunks)


IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [34]:
len(result_chunks)

97

In [35]:
len(chunks)

129

In [36]:
from datetime import datetime

# get the current date
current_date = datetime.now().date()

print("Current date:", current_date)


Current date: 2024-06-13


In [37]:
#make sure directory path exists
import os

def ensure_directory_exists(path):
    if not os.path.exists(path):
        os.makedirs(path)
        print(f"Made directory path: {path}")
    else:
        print(f"Directory path already exists: {path}")

# usage before saving the file
directory_path = f"/Users/rileybitterli/Desktop/StockBot_Data/Draft_VIII/Pickle_Files/{year}/{quarter}"
ensure_directory_exists(directory_path)

Directory path already exists: /Users/rileybitterli/Desktop/StockBot_Data/Draft_VIII/Pickle_Files/2014/QTR1


In [38]:
 send_email('its me', 'starting scraping')

In [ ]:
# heavy lifter of those notebook- open up each of the links, navigate it, rip what we need, store in pickle file every 1000

import threading
from selenium import webdriver
from selenium.webdriver.common.by import By
from time import sleep
from random import uniform
import pandas as pd
import pickle
import concurrent.futures


# for thread safety
batch_number_counter = [highest_batch]  # use a list so it's mutable
batch_number_lock = threading.Lock()

def scrape_chunk(chunk_number, chunk):
    global batch_number_counter
    necessary_dfs = []  
    # setup a new webdriver instance
    driver = webdriver.Chrome(service=s, options = chrome_options)
    driver.delete_all_cookies()
    index = 0
    url_counter = 0
    with batch_number_lock:
        batch_number = batch_number_counter[0]
        batch_number_counter[0] += 1

    for url, date in chunk:
        index += 1
        try: 
            sec = uniform(.4,1)
            print("I slept for:", round(sec, 2), "seconds, and my index is:", index, "url counter is:", url_counter, 'batch number', batch_number, "chunk_number:", chunk_number)
            driver.get(url)
            sleep(sec)
            web_elements = driver.find_elements(By.TAG_NAME, "a")
            links = [we.get_attribute("href") for we in web_elements if we.get_attribute("href")]
            html_links = [link for link in links if 'xml' in link]
    
            if html_links:
                driver.get(html_links[0])
                full_page = driver.page_source.replace('%', '')
                dfs = pd.read_html(full_page)

        
                for df in dfs:
                   if isinstance(df, pd.DataFrame):
                    try:
                        # ensure we're working with a string for the regex search
                        first_item = str(df.iloc[0, 1]) if not df.empty and df.columns.size > 1 else ""
                        bracket_match = re.search(r'\[(.*?)\]', first_item)
                        if bracket_match:
                            ticker = bracket_match.group(1)  # Extract the value between the brackets
                            name = df.iloc[1, 1]
                    except Exception as e:
                        # if any error occurs, ignore and move on
                        pass 

                    if isinstance(df.columns, pd.MultiIndex):
                        for col in df.columns.levels[0]:
                            if col.startswith('Table I - Non-Derivative Securities Acquired, Disposed of, or Beneficially Owned'):
                                df.columns = df.columns.droplevel(0).droplevel(0)
                                df.rename(columns={
                                        '1. Title of Security (Instr. 3)': 'security_type',
                                        '2. Transaction Date  (Month/Day/Year)': 'date',
                                        '2A. Deemed Execution Date, if any  (Month/Day/Year)': 'execution_date',
                                        'Code': 'code',
                                        'V': 'v',
                                        '(A) or (D)': 'a_or_d',
                                        'Amount': 'amount',
                                        '5. Amount of Securities Beneficially Owned Following Reported Transaction(s) (Instr. 3 and 4)': 'total_owned_after_trans',
                                        '6. Ownership Form: Direct (D) or Indirect (I) (Instr. 4)': 'direct_or_indirect'
                                    }, inplace=True)

                                df['URL'] = url
                                df['Filing Date'] = date
                                df['Ticker'] = ticker
                                df['Name'] = name

                                if 'code' in df.columns and (df['code'].eq('P').any() or df['code'].eq('S').any()):
                                    necessary_dfs.append(df)
                                        
                                        
                                        
                url_counter += 1
                if url_counter == 1000:
                     with open(f"/Users/rileybitterli/Desktop/StockBot_Data/Draft_VIII/Pickle_Files/{year}/{quarter}/{current_date}_batch{batch_number}.pkl", "wb") as file:
                        pickle.dump(necessary_dfs, file)
                        necessary_dfs = []
                        url_counter = 0
                        
                        
        except IndexError as ie:
            print(f"Skipped URL due to error at index:{index}. Error: {str(ie)}")
            continue
                        
        except ValueError:
            continue

        except Exception as e:
            if "Document is empty" in str(e):
                print(f"Skipped URL due to error at index:{index}. Error: {str(e)}")
                continue
            else:
                send_email('Error in Jupyter Notebook', f"Broke at index:{index} An error occurred: {str(e)}")
                raise e

    if necessary_dfs:
        with open(f"/Users/rileybitterli/Desktop/StockBot_Data/Draft_VIII/Pickle_Files/{year}/{quarter}/{current_date}_batch{batch_number}.pkl", "wb") as file:
            pickle.dump(necessary_dfs, file)
            necessary_dfs = []
            url_counter = 0
    print('all done')
    driver.close()

if __name__ == '__main__':
    with concurrent.futures.ThreadPoolExecutor(max_workers=8) as executor:
        futures = [executor.submit(scrape_chunk, chunk[0], chunk[1]) for chunk in result_chunks]
        concurrent.futures.wait(futures)


In [ ]:
 send_email('its me', f'done with {year} {quarter}')